(Material adapted from Luis Fernando Laris Pardo's)
# Autoencoders
For today's exercise we are going to be implementing autoencoders. One cool implementation can be found [here](https://medium.com/@sorenlind/a-deep-convolutional-denoising-autoencoder-for-image-classification-26c777d3b88e) where autoencoders are used to get unnoisy images from magic the gathering cards. 

For this I have used the most common starter example for autoencoders that can be found using the mnist dataset. Remember that the autoencoder is doing something similar to the following:

![image](https://www.pyimagesearch.com/wp-content/uploads/2020/02/keras_denoising_autoencoder_overview.png)

As in previous weeks, most of the code is already given.

**Your Task**:

* Add artificial noise to the input images and see how your autoencoder works with these.
* Add more dense layers to your autoencoder model

**(Very) Optional Tasks:** Take a look at the articles at bottom page if you are going to work on this topic for your exam project.

* Apply *[Image data augmentation](https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/)* to the dataset to improve the quality of the autoencoder
* Create a *[CNN-based autoencoder](https://towardsdatascience.com/convolutional-autoencoders-for-image-noise-reduction-32fce9fc1763)*: Use convolutional, maxpooling and upscaling layers

In [ ]:
from tensorflow.keras.datasets import mnist
import numpy as np
from tensorflow.keras.layers import Input, Dense, Flatten, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adadelta, SGD, Adam
import matplotlib.pyplot as plt

### Read the data and flatten it
In this case, first we are going to use dense layers, here is another way to flattern your data before giving it to the dense layer.

In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()

In [ ]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# flatten the data (if working with Dense Network)

x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:]))) 
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Add Noise to x_train, x_test
x_train_noisy = x_train + np.random.normal(loc=0.0, scale=0.1, size=x_train.shape)
x_test_noisy = x_test + np.random.normal(loc=0.0, scale=0.1, size=x_test.shape)

# OPTIONAL: Data Augmentation - Transform the data (random translation/rotation/scale/shear/...)
# YOUR CODE HERE

print(x_train.shape)
print(x_test.shape)

### Create your autoencoder
We can now create the model for the autoencoder, in this case is a very simple one where of an input layer, one hiden layer (the encoded part) and an output later (the decoded part).

In [ ]:
# this is the size of our encoded representations
encoding_dim = 32

input_img = Input(shape=(784,))

encoded = Dense(encoding_dim, activation='relu')(input_img)
# You can add multiple layers (of any type) in the following way, but remember to always create an "hour-glass-shaped" network:
# encoded = Dense(<N_NEURONS>, activation=<ACTIVATION>)(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

### The encoder and decoder models
This models we are going to use them later to test the results.


In [ ]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

### Time to train our model
Try different optimizers, deffinitely SGD is not the best for this task. you should be able to ger good results with the setup given

In [ ]:
opt = Adam()
autoencoder.compile(optimizer=opt, loss='binary_crossentropy')

In [ ]:
# Use the noisy data as your input and the non-noisy data as the ground truth
autoencoder.fit(x_train_noisy, x_train,
                epochs=3, # CHANGE ME, PLIZ
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

### Now let's test our Autoencoder!
One important thing to notice is how reshape is used to rearrange the image plot it.

In [ ]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

In [ ]:
n = 10  
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# Optional Articles

### Image Data Augmentation
https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/
### Convolutional Autoencoders for Image Noise Reduction
https://towardsdatascience.com/convolutional-autoencoders-for-image-noise-reduction-32fce9fc1763

# Extra Articles

### Anomaly Detection with Autoencoders
https://towardsdatascience.com/anomaly-detection-with-autoencoder-b4cdce4866a6